# DL model with MNIST Lesson 4 page 118 - 128

also used https://caffe2.ai/docs/tutorial-MNIST.html since there was some duplication in the pdf's code

In [1]:
%matplotlib inline
from matplotlib import pyplot
import numpy as np
import os
import shutil
import caffe2.python.predictor.predictor_exporter as pe

from caffe2.python import (
    brew,
    core,
    model_helper,
    net_drawer,
    optimizer,
    visualize,
    workspace)

core.GlobalInit(['caffe2', '--caffe2_log_level=0'])
print ("Necessities imported!")

USE_LENET_MODEL = True

ModuleNotFoundError: No module named 'caffe2'

In [ ]:
from keras.datasets.mnist import load_data

(trainX, trainy), (testX, testy) = load_data()

print ('Train', trainX.shape, trainy.shape)
print ('Test', testX.shape, testy.shape)

In [ ]:
'''
# Commented out some of  the following sections since I manually downloaded and unzipped the files
# due to StringIO issues with python3. Following the rest of the processes/commenting them out as needed

def DownloadResource(url, path):
    # Downloads resources from s3 by url and unzips them to the provided path
    
    import requests, zipfile, io
    from io import BytesIO           # changed for python3 
    print ("Downloading... {} to {}".format(url, path))
    r = requests.get(url, stream=True)
    z = zipfile.ZipFile(np.genfromtxt(io.BytesIO(r.encode())))
    # z = zipfile.ZipFile(io.StringIO(r.content))
    z.extractall(path)
    print("Completed download and extraction.")
'''

In [ ]:
current_folder = os.path.join(os.path.expanduser('~'),
                              'Desktop\Education\LiveClasses\DeepLearningKandTF\caffe2_notebooks')
data_folder = os.path.join(current_folder, 'tutorial_data', 'mnist')
root_folder = os.path.join(current_folder, 'tutorial_files', 'tutorial_mnist')
db_missing = False

In [ ]:
if not os.path.exists(data_folder):
    os.makedirs(data_folder)   
    print("Your data folder was not found!! This was generated: {}".format(data_folder))

In [ ]:
if os.path.exists(os.path.join(data_folder, "mnist-train-nchw-lmdb")):
    print ("lmdb train db found")
else:
    db_missing = True

In [ ]:
if os.path.exists(os.path.join(data_folder,"mnist-test-nchw-lmdb")):
    print("lmdb test db found!")
else:
    db_missing = True

In [ ]:
'''
# commented out as stated above

# attempt the download of the db if either was missing
if db_missing:
    print("one or both of the MNIST lmbd dbs not found!!")
    db_url = "http://download.caffe2.ai/databases/mnist-lmdb.zip"
    try:
        DownloadResource(db_url, data_folder)
    except Exception as ex:
        print("Failed to download dataset. Please download it manually from {}".format(db_url))
        print("Unzip it and place the two database folders here: {}".format(data_folder))
        raise ex
'''

In [ ]:
# clean up the statistics from any of the old runs

if os.path.exists(root_folder):
    print ("Looks like you ran this before, so we need to cleanup those old files...")
    shutil.rmtree(root_folder)
    
os.makedirs(root_folder)
workspace.ResetWorkspace(root_folder)

print("training data folder:" + data_folder)
print("workspace root folder:" + root_folder)

In [ ]:
def AddInput(model, batch_size, db, db_type):
    
    # load the data
    data_uint8, label = model.TensorProtosDBInput(
        [], ["data_uint8", "label"], batch_size=batch_size,
        db=db, db_type=db_type)
    
    # cast the data to float
    data = model.Cast(data_uint8, "data", to=core.DataType.FLOAT)
    
    # scale data from [0,255] down to [0,1]
    data = model.Scale(data, data, scale=float(1./256))
    
    # don't need the gradient for the backward pass
    data = model.StopGradient(data, data)
    return data, label

In [ ]:
def AddLeNetModel(model, data):
    '''
    This part is the standard LeNet model: from data to the softmax prediction.

    For each convolutional layer we specify dim_in - number of input channels
    and dim_out - number or output channels. Also each Conv and MaxPool layer changes the
    image size. For example, kernel of size 5 reduces each side of an image by 4.

    While when we have kernel and stride sizes equal 2 in a MaxPool layer, it divides
    each side in half.
    '''
    
    # Image size: 28 x 28 -> 24 x 24
    conv1 = brew.conv(model, data, 'conv1', dim_in=1, dim_out=20, kernel=5)
    
    # Image size: 24 x 24 -> 12 x 12
    pool1 = brew.max_pool(model, conv1, 'pool1', kernel=2, stride=2)
    
    # Image size: 12 x 12 -> 8 x 8
    conv2 = brew.conv(model, pool1, 'conv2', dim_in=20, dim_out=100, kernel=5)
    
    # Image size: 8 x 8 -> 4 x 4
    pool2 = brew.max_pool(model, conv2, 'pool2', kernel=2, stride=2)
    
    # 50 * 4 * 4 stands for dim_out from previous layer multiplied by the image size
    fc3 = brew.fc(model, pool2, 'fc3', dim_in=100 * 4 * 4, dim_out=500)

    #fc3 = brew.relu(model, fc3, fc3)
    relu = brew.relu(model, fc3, fc3)
    pred = brew.fc(model, relu, 'pred', 500, 10)
    softmax = brew.softmax(model, pred, 'softmax')
    
    return softmax

In [ ]:
def AddAccuracy(model, softmax, label):
    # Adds an accuracy op to the model
    accuracy = brew.accuracy(model, [softmax, label], "accuracy")
    
    return accuracy

In [ ]:
def AddTrainingOperators(model, softmax, label):
    # Adds training operators to the model.
    
    xent = model.LabelCrossEntropy([softmax, label], 'xent')
    
    # compute the expected loss
    loss = model.AveragedLoss(xent, "loss")
    
    # track the accuracy of the model
    AddAccuracy(model, softmax, label)
    
    # use the average loss we just computed to add gradient operators to the model
    model.AddGradientOperators([loss])
    
    # do a simple stochastic gradient descent
    ITER = brew.iter(model, "iter")
    
    # set the learning rate schedule
    LR = model.LearningRate(
        ITER, "LR", base_lr=-0.1, policy="step", stepsize=1, gamma=0.999 )
    
    # ONE is a constant value that is used in the gradient update. We only need
    # to create it once, so it is explicitly placed in param_init_net.
    ONE = model.param_init_net.ConstantFill([], "ONE", shape=[1], value=1.0)
    
    # Now, for each parameter, we do the gradient updates.
    for param in model.params:
        
        # Note how we get the gradient of each parameter - ModelHelper keeps
        # track of that.
        param_grad = model.param_to_grad[param]
        
        # The update is a simple weighted sum: param = param + param_grad * LR
        model.WeightedSum([param, ONE, param_grad, LR], param)
        
    '''
    We will need to checkpoint the parameters of the model periodically. This is 
    achieved via the Checkpoint operator. It also takes in a parameter “every” 
    so that we don’t checkpoint way too often. In this case, we will say let’s 
    checkpoint every 20 iterations, which should probably be fine.
    # not entered in code so I commented this out
    
    model.Checkpoint([ITER] + model.params, [],
               db="mnist_lenet_checkpoint_%05d.lmdb",
               db_type="lmdb", every=20)
               
               
    # The below is the pdf version of setting up the model after the average loss 
    # was added
    optimizer.build_sgd(model, base_learning_rate=0.1, 
                        policy='step', stepsize=1, gamma=0.999)
    '''

In [ ]:
def AddBookkeepingOperators(model):
    """This adds a few bookkeeping operators that we can inspect later.

    These operators do not affect the training procedure: they only collect
    statistics and prints them to file or to logs.
    """
    
    # Print basically prints out the content of the blob. to_file=1 routes the
    # printed output to a file. The file is going to be stored under
    #     root_folder/[blob name]
    model.Print('accuracy', [], to_file=1)
    model.Print('loss', [], to_file=1)
    
    # Summarizes the parameters. Different from Print, Summarize gives some
    # statistics of the parameter, such as mean, std, min and max.
    for param in model.params:
        model.Summarize(param, [], to_file=1)
        model.Summarize(model.param_to_grad[param], [], to_file=1)
        
    # Now, if we really want to be verbose, we can summarize EVERY blob
    # that the model produces; it is probably not a good idea, because that
    # is going to take time - summarization do not come for free. For this
    # demo, we will only show how to summarize the parameters and their
    # gradients.

In [ ]:
# This part is in the pdf but not the site. I'm following the site so commented the below out

'''
def AddMLPModel(model, data):
    size = 28 * 28 * 1
    sizes = [size, size * 2, size * 2, 10]
    layer = data
    for i in range(len(sizes) - 1):
        layer = brew.fc(model, layer, 'dense_{}'.format(i), dim_in=sizes[i],
                        dim_out=sizes[i + 1])
        layer = brew.relu(model, layer, 'relu_{}'.format(i))
        softmax = brew.softmax(model,layer, 'softmax')
        
        return softmax
'''

In [ ]:
# Another part from the pdf. Seems like the above section was meant to allow for easy
# switching between 2 different models

'''
def AddModel(model, data):
    if USE_LENET_MODEL:
        return AddLeNetModel(model, data)
    else:
        return AddMLPModel(model, data)
'''

In [ ]:
arg_scope = {"order": "NCHW"}
train_model = model_helper.ModelHelper(name="mnist_train", arg_scope=arg_scope)
data, label = AddInput(train_model, batch_size=64,
                       db=os.path.join(data_folder, 'mnist-train-nchw-lmdb'),
                       db_type='lmdb')
softmax = AddLeNetModel(train_model, data)
AddTrainingOperators(train_model, softmax, label)
AddBookkeepingOperators(train_model)

In [ ]:
# Testing model. We will set the batch size to 100, so that the testing
# pass is 100 iterations (10,000 images in total).
# For the testing model, we need the data input part, the main LeNetModel
# part, and an accuracy part. Note that init_params is set False because
# we will be using the parameters obtained from the train model.
test_model = model_helper.ModelHelper(name="mnist_test", 
                                      arg_scope=arg_scope, init_params=False)
data, label = AddInput(test_model, batch_size=100,
                       db=os.path.join(data_folder, 'mnist-test-nchw-lmdb'),
                       db_type='lmdb')
softmax = AddLeNetModel(test_model, data)
AddAccuracy(test_model, softmax, label)

In [ ]:
# Deployment model. We simply need the main LeNetModel part.
deploy_model = model_helper.ModelHelper(name="mnist_deploy", 
                                        arg_scope=arg_scope, init_params=False)
AddLeNetModel(deploy_model, "data")

# You may wonder what happens with the param_init_net part of the deploy_model.
# No, we will not use them, since during deployment time we will not randomly
# initialize the parameters, but load the parameters from the db.

In [ ]:
# from the site. for visualization. doesn't work due to a problem with pydot or something. commented out
'''
from IPython import display
graph = net_drawer.GetPydotGraph(train_model.net.Proto().op, "mnist", rankdir="LR")
display.Image(graph.create_png(), width=800)
'''

In [ ]:
'''
same as above

graph = net_drawer.GetPydotGraphMinimal(train_model.net.Proto().op, "mnist", 
                                        rankdir="LR", minimal_dependency=True)
display.Image(graph.create_png(), width=800)
'''

In [ ]:
print(str(train_model.param_init_net.Proto())[:400] + '\n...')

In [ ]:
with open(os.path.join(root_folder, "train_net.pbtxt"), 'w') as fid:
    fid.write(str(train_model.net.Proto()))
with open(os.path.join(root_folder, "train_init_net.pbtxt"), 'w') as fid:
    fid.write(str(train_model.param_init_net.Proto()))
with open(os.path.join(root_folder, "test_net.pbtxt"), 'w') as fid:
    fid.write(str(test_model.net.Proto()))
with open(os.path.join(root_folder, "test_init_net.pbtxt"), 'w') as fid:
    fid.write(str(test_model.param_init_net.Proto()))
with open(os.path.join(root_folder, "deploy_net.pbtxt"), 'w') as fid:
    fid.write(str(deploy_model.net.Proto()))
print("Protocol buffers files have been created in your root folder: " + root_folder)

In [ ]:
# The parameter initialization network only needs to be run once.
workspace.RunNetOnce(train_model.param_init_net)

# creating the network
workspace.CreateNet(train_model.net, overwrite=True)

# set the number of iterations and track the accuracy & loss
total_iters = 200
accuracy = np.zeros(total_iters)
loss = np.zeros(total_iters)

# Now, we will manually run the network for 200 iterations.
for i in range(total_iters):
    workspace.RunNet(train_model.net)
    accuracy[i] = workspace.FetchBlob('accuracy')
    loss[i] = workspace.FetchBlob('loss')
    #this if statement is from pdf
    if i % 25 == 0:
        print("Iter: {}, Loss: {}, Accuracy: {}".format(i, loss[i], accuracy[i]))
    
# After the execution is done, let's plot the values.
pyplot.plot(loss, 'b')
pyplot.plot(accuracy, 'r')
pyplot.title("Summary of Training Run")
pyplot.xlabel("Iteration")
pyplot.legend(('Loss', 'Accuracy'), loc='upper right')

In [ ]:
# Let's look at some of the data.
pyplot.figure()
data = workspace.FetchBlob('data')
_ = visualize.NCHW.ShowMultiple(data)
pyplot.figure()
softmax = workspace.FetchBlob('softmax')
_ = pyplot.plot(softmax[0], 'ro')
pyplot.title('Prediction for the first image')

In [ ]:
# Convolutions for this mini-batch
pyplot.figure()
conv = workspace.FetchBlob('conv1')
shape = list(conv.shape)
shape[1] = 1
# We can look into any channel. This of it as a feature model learned
conv = conv[:,15,:,:].reshape(shape)

_ = visualize.NCHW.ShowMultiple(conv)

In [ ]:
# run a test pass on the test net
workspace.RunNetOnce(test_model.param_init_net)
workspace.CreateNet(test_model.net, overwrite=True)
test_accuracy = np.zeros(100)
for i in range(100):
    workspace.RunNet(test_model.net.Proto().name)
    test_accuracy[i] = workspace.FetchBlob('accuracy')
# After the execution is done, let's plot the values.
pyplot.plot(test_accuracy, 'r')
pyplot.title('Acuracy over test batches.')
print('test_accuracy: %f' % test_accuracy.mean())

In [ ]:
# construct the model to be exported
# the inputs/outputs of the model are manually specified.
pe_meta = pe.PredictorExportMeta(
    predict_net=deploy_model.net.Proto(),
    parameters=[str(b) for b in deploy_model.params],
    inputs=["data"],
    outputs=["softmax"],
)

# save the model to a file. Use minidb as the file format
pe.save_to_db("minidb", os.path.join(root_folder, "mnist_model.minidb"), pe_meta)
print("The deploy model is saved to: " + root_folder + "/mnist_model.minidb")


In [ ]:
# we retrieve the last input data out and use it in our prediction test before we scratch the workspace
blob = workspace.FetchBlob("data")
pyplot.figure()
_ = visualize.NCHW.ShowMultiple(blob)

# reset the workspace, to make sure the model is actually loaded
workspace.ResetWorkspace(root_folder)

# verify that all blobs are destroyed.
print("The blobs in the workspace after reset: {}".format(workspace.Blobs()))

# load the predict net
predict_net = pe.prepare_prediction_net(os.path.join(root_folder, "mnist_model.minidb"), "minidb")

# verify that blobs are loaded back
print("The blobs in the workspace after loading the model: {}".format(workspace.Blobs()))

# feed the previously saved data to the loaded model
workspace.FeedBlob("data", blob)

# predict
workspace.RunNetOnce(predict_net)
softmax = workspace.FetchBlob("softmax")

# the first letter should be predicted correctly
pyplot.figure()
_ = pyplot.plot(softmax[0], 'ro')
pyplot.title('Prediction for the first image')